# New York City Property Sales Analysis - Data Preparation

On this notebook, all the preprocess required on the original dataset is proposed, including the geocoding step, where the information about the location of the properties is converted to geographical coordinates to be used later on map visualizations. 

On the end of this notebook, two datasets are created and exported:
* The first one will be used on a Exploratory Data Analysis
* And the last one is proposed to be used on a Streamlit App Development.

In [1]:
import numpy as np 
from datetime import datetime
import pandas as pd 
import geocoder

## Read Data

First off, the original data is readed.

In [2]:
data = pd.read_csv('nyc-rolling-sales.csv')

In [3]:
data.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


## Geocoding

On this section, the geocoding process is proposed. For this, the geocoder library is used to convert the Zip Codes into Geographical Coordinates. It is important to note that the ideal way of doing this step would be to use the entire address field as a query to obtain the coordinates, but that would require the conversion of much more rows and take too much time. So, for simplicity, only the unique values of Zip Code (extracted below) are used on this process, and the results are them passed to the full dataset by a Join Operation.

In [4]:
zip_codes = data['ZIP CODE'].unique()

Below, a iteractive process is proposed to query every row of the Zip Codes used and obtain the coordinates.

In [5]:
x_coordinates = []
y_coordinates = []
i = 0
for codes in zip_codes:
    g = geocoder.osm(str(codes) + ', New York')
    if g.ok == True:
        x_coordinates.append(g.osm['x'])
        y_coordinates.append(g.osm['y'])
    else:
        x_coordinates.append('Not Found')
        y_coordinates.append('Not Found')

The resulted coordinates are them combined with the respective Zip Codes on a new dataframe.

In [6]:
geocode_df = pd.DataFrame(list(zip(zip_codes,x_coordinates,y_coordinates)),columns=['Zip Codes','X Coordinate','Y Coordinate'])
geocode_df.head()

,Zip Codes,X Coordinate,Y Coordinate
0,10009,-74.006015,40.712728
1,0,-73.498902,42.671362
2,10002,-74.006015,40.712728
3,10011,-74.006015,40.712728
4,10001,-74.006015,40.712728


Finally, the resulted dataframe is merged with the original data by a Left Join.

In [7]:
data = data.merge(geocode_df,how='left',left_on='ZIP CODE',right_on='Zip Codes')

In [8]:
data = data.drop('Zip Codes',axis=1)

In [9]:
data = data[data['X Coordinate'] != 'Not Found']

In [10]:
data['X Coordinate'] = data['X Coordinate'].astype(float)
data['Y Coordinate'] = data['Y Coordinate'].astype(float)

## Pre-processing

On this section, some preparation of the data is proposed, which is important both for the Exploratory Analysis and the Streamlit app. First off, unused columns are removed from the dataset.

In [11]:
data = data.drop(['Unnamed: 0','BLOCK','LOT','APARTMENT NUMBER'],axis=1)

After that, some variables used have their format changed for a better performance.

In [12]:
data[['SALE PRICE','LAND SQUARE FEET','GROSS SQUARE FEET']] = data[['SALE PRICE','LAND SQUARE FEET','GROSS SQUARE FEET']].replace({' -  ':'0'})
data[['SALE PRICE','LAND SQUARE FEET','GROSS SQUARE FEET']] = data[['SALE PRICE','LAND SQUARE FEET','GROSS SQUARE FEET']].astype('float64')

In [13]:
data['SALE DATE'] = pd.to_datetime(data['SALE DATE'])

In the Sales Price column, some columns appear to be Outliers. To remove any possible invalid data, properties with price greater than $5000000 are removed from the dataset. Also, a great number of Properties have a null Price, which corresponds to transfers of ownerships, as stated by the documentations of the dataset. This data is also removed from the dataset.

In [14]:
data = data[(data['SALE PRICE'] > 10000) & (data['SALE PRICE'] < 5000000)]

Also on the Year Built Column, some strange values are removed.

In [15]:
data = data[data['YEAR BUILT'] > 0]

The Borough column is codified. According to the description of the data provided at Kaggle, the possible values are Manhattan (1), Bronx (2), Brooklyn (3), Queens (4), and Staten Island (5). Let's configure that on data.

In [16]:
data['BOROUGH'] = data['BOROUGH'].replace({1:'Manhattan', 2:'Bronx', 3:'Brooklyn', 4:'Queens', 5:'Staten Island'})

Finally, just for aestethics reason, let's format some columns with the Title Method.

In [17]:
data['BUILDING CLASS CATEGORY'] = data['BUILDING CLASS CATEGORY'].str.title()
data['NEIGHBORHOOD'] = data['NEIGHBORHOOD'].str.title()
data['ADDRESS'] = data['ADDRESS'].str.title()

The Title Method is also applied to the column names.

In [18]:
data.columns = data.columns.str.title()

## Export Data

Finally, on this last section, the resulted data are exported to further use on this project.

### Notebook Data

With no required changes, the resulted dataframe is exported to be used on Exploratory Analysis Notebook.

In [19]:
data.to_csv('EDA/nyc-notebook.csv')

### Streamlit App Data

Here, some further processign is made to obtain the data used on the Streamlit App.

In [20]:
data.head()

,Borough,Neighborhood,Building Class Category,Tax Class At Present,Ease-Ment,Building Class At Present,Address,Zip Code,Residential Units,Commercial Units,Total Units,Land Square Feet,Gross Square Feet,Year Built,Tax Class At Time Of Sale,Building Class At Time Of Sale,Sale Price,Sale Date,X Coordinate,Y Coordinate
3,Manhattan,Alphabet City,07 Rentals - Walkup Apartments,2B,,C4,154 East 7Th Street,10009,10,0,10,2272.0,6794.0,1913,2,C4,3936272.0,2016-09-23,-74.006015,40.712728
6,Manhattan,Alphabet City,07 Rentals - Walkup Apartments,2B,,C4,210 Avenue B,10009,8,0,8,1750.0,4226.0,1920,2,C4,3192840.0,2016-09-23,-74.006015,40.712728
13,Manhattan,Alphabet City,09 Coops - Walkup Apartments,2,,C6,"327 East 3 Street, 1C",10009,0,0,0,0.0,0.0,1920,2,C6,499000.0,2017-03-10,-74.006015,40.712728
15,Manhattan,Alphabet City,09 Coops - Walkup Apartments,2,,C6,"327 East 3Rd Street, 5A",10009,0,0,0,0.0,0.0,1920,2,C6,529500.0,2017-06-09,-74.006015,40.712728
16,Manhattan,Alphabet City,09 Coops - Walkup Apartments,2,,C6,"327 East 3 Street, 2E",10009,0,0,0,0.0,0.0,1920,2,C6,423000.0,2017-07-14,-74.006015,40.712728


Here, the numerical and categorical data are grouped according to Zip Codes. Since the Numerical data is aggregated by the Mean Operation, and the Categorical data is aggregated by the most common category, two differente dataframes are created.

In [21]:
numerical_app_data = data.groupby('Zip Code').mean()
numerical_app_data = numerical_app_data.reset_index()

In [22]:
categorical_app_data = data[['Zip Code','Building Class Category','Address','Borough','Neighborhood']].groupby('Zip Code').agg(lambda x: x.value_counts().index[0])
categorical_app_data = categorical_app_data.reset_index()

The resulted dataframes are them joined together and exported as a csv file.

In [23]:
app_data = numerical_app_data.merge(categorical_app_data,how='left',on='Zip Code')

In [24]:
app_data.to_csv('App/nyc-app.csv')